# Visualizations

## Setup

In [ ]:
import matplotlib.pyplot as plt
from herbie import FastHerbie, paint, wgrib2, Herbie
# from herbie.toolbox import EasyMap, ccrs, pc
import xarray as xr
from datetime import datetime
import sys
import os.path as osp
import pandas as pd
import numpy as np
sys.path.append("../src")
from utils import Dict, read_yml, str2time, print_dict_summary, read_pkl
import ingest.retrieve_hrrr_api as ih
import ingest.retrieve_raws_stash as rrs
from viz import map_var, create_gif

In [ ]:
with open("../etc/forecast_config.json", "r") as json_file:
    config = json.load(json_file)   
    config = Dict(config)
config.update({
    'start_time': "2024-12-01T00:00:00Z",
    'end_time': "2024-12-02T23:00:00Z"
})
print_dict_summary(config)

In [ ]:
## Retrieve Data
# hrrr_ds = ih.retrieve_hrrr(config)
# hrrr_ds["date_time"] = hrrr_ds["date_time"].astype("datetime64[ns]")

## Save Data
# hrrr_ds.to_netcdf("../data/hrrr_ds.nc")

## Read HRRR Data
hrrr_ds = xr.open_dataset("../data/hrrr_ds.nc")

In [ ]:
config.update({
    'start_time': "2023-12-01T00:00:00Z",
    'end_time': "2023-12-07T23:00:00Z"
})
raws_dict = rrs.build_raws_dict(config)

## Viz

### Maps

In [ ]:
H = Herbie("2024-01-01", product="sfc")
ds = H.xarray("(?:HGT|LAND):surface")

hrrr_ds["Ew"] = hrrr_ds.Ew.where(ds.lsm > 0)

In [ ]:
mt_time = pd.to_datetime(hrrr_ds.valid_time.values).tz_localize("UTC").tz_convert("America/Denver")

In [ ]:
tstep = 0

t = hrrr_ds.valid_time[tstep]
t = f"{t.dt.year.item():04d}-{t.dt.month.item():02d}-{t.dt.day.item():02d} {t.dt.hour.item():02d}:{t.dt.minute.item():02d}:{t.dt.second.item():02d}"

map_var(hrrr_ds, "Ew", 
        time_step = tstep,
        legend_title = "Fuel Moisture Content (%)",
        title = f"FMC Forecast at {t} UTC"
       )

In [ ]:
# Define time steps (e.g., first 10)
time_steps = range(0, 48)  # Adjust as needed
create_gif(hrrr_ds, "Ew", time_steps, gif_path='../outputs/test_forecast.gif', duration=0.5)


### Timeseries Plots

In [ ]:
st = "TT562"
y = raws_dict[st]["RAWS"]["fm"].to_numpy()
x = raws_dict[st]["times"]
plt.plot(x, y, linestyle='-',c='#468a29',label='FM Observed')
plt.legend()
plt.title(f"Observed FMC at RAWS {st}")
plt.xticks(rotation=90)
plt.savefig("../outputs/fmc_ts.png")

In [ ]:
# Calculate autocorrelations
y2 = pd.Series(y)
lags = range(0, 54+1)  # Specify the range of lags
autocorrelations = [y2.autocorr(lag=lag) for lag in lags]

# Plot autocorrelation
default_blue = plt.rcParams['axes.prop_cycle'].by_key()['color'][0]
plt.bar(lags, autocorrelations)
plt.xlabel('Lag Hour')
plt.ylabel('Autocorrelation')
plt.title(f'Autocorrelation Plot - RAWS {st}')
for lag in [12, 24, 36, 48]:
    plt.axvline(x=lag, color=default_blue, linestyle='--')
    plt.text(x=lag, y=0.8, s=f'Lag {lag}', color=default_blue, 
             rotation=90, va='center', ha='right', fontweight='bold')  # Bold text

plt.savefig("../outputs/fmc_acf.png")
plt.show()

In [ ]:
# White noise process
np.random.seed(42)  # Set seed for reproducibility
x = np.arange(1, 1001)  # Time steps from 1 to 100
y = np.random.normal(loc=0, scale=1, size=1000)  # Gaussian noise with mean 0, SD 1

# Plot the white noise process
# plt.figure(figsize=(10, 5))
# plt.plot(x, y, marker='o', linestyle='-', label='White Noise')
plt.plot(x, y)
plt.title("White Noise Process - Normal(0,1)")
plt.xlabel("Time Step")
plt.ylabel("Value")
plt.axhline(y=0, color='gray', linestyle='--', linewidth=0.8)  # Add horizontal line at y=0
plt.grid(alpha=0.3)
plt.savefig("../outputs/white_noise.png")
plt.show()